Методы и алгоритмы обработки слабоструктурированных данных
Лабораторная работа №2 (исправленная)
Болотов М.В. АСУ4-22-1м

In [1]:
import csv
import json  
from bs4 import BeautifulSoup                                                       
import xml.etree.ElementTree as ET

def load_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def save_to_file(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(data)

def detect_input_format(file_path):
    if file_path.endswith('.html'):
        return 'html'
    elif file_path.endswith('.xml'):
        return 'xml'
    elif file_path.endswith('.json'):
        return 'json'
    elif file_path.endswith('.csv'):
        return 'csv'
    else:
        return None

def detect_output_format(output_file_path):
    if output_file_path.endswith('.html'):
        return 'html'
    elif output_file_path.endswith('.xml'):
        return 'xml'
    elif output_file_path.endswith('.json'):
        return 'json'
    elif output_file_path.endswith('.csv'):
        return 'csv'
    else:
        return None

def html_to_json(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    return json.dumps(soup.prettify(), indent=4)

def html_to_xml(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    return soup.prettify()

def xml_to_json(xml_string):
    root = ET.fromstring(xml_string)
    xml_dict = {root.tag: traverse_xml(root)}
    return json.dumps(xml_dict, indent=4)

def xml_to_csv(xml_string):
    root = ET.fromstring(xml_string)
    csv_data = []
    for child in root:
        row = [child.tag] + [elem.text for elem in child]
        csv_data.append(row)
    return '\n'.join([','.join(row) for row in csv_data])

def json_to_xml(json_string):
    json_dict = json.loads(json_string)
    root_tag = list(json_dict.keys())[0]
    root = ET.Element(root_tag)
    build_xml(root, json_dict[root_tag])
    return ET.tostring(root, encoding='unicode')

def json_to_csv(json_string):
    json_dict = json.loads(json_string)
    header = list(json_dict[0].keys())
    rows = [header]
    for item in json_dict:
        rows.append([str(item.get(key, '')) for key in header])
    return '\n'.join([','.join(row) for row in rows])

def csv_to_json(csv_string):
    rows = [row.split(',') for row in csv_string.split('\n')]
    header = rows[0]
    json_data = []
    for row in rows[1:]:
        json_data.append({header[i]: value for i, value in enumerate(row)})
    return json.dumps(json_data, indent=4)

def csv_to_xml(csv_string):
    rows = [row.split(',') for row in csv_string.split('\n')]
    root = ET.Element('root')
    for row in rows:
        parent = ET.SubElement(root, row[0])
        for value in row[1:]:
            child = ET.SubElement(parent, 'item')
            child.text = value
    return ET.tostring(root, encoding='unicode')

def traverse_xml(element):
    xml_dict = {}
    for child in element:
        if child.tag in xml_dict:
            if isinstance(xml_dict[child.tag], list):
                xml_dict[child.tag].append(traverse_xml(child))
            else:
                xml_dict[child.tag] = [xml_dict[child.tag], traverse_xml(child)]
        else:
            xml_dict[child.tag] = traverse_xml(child) if len(child) > 0 else child.text
    return xml_dict

def build_xml(parent, data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, list):
                for item in value:
                    child = ET.SubElement(parent, key)
                    build_xml(child, item)
            else:
                child = ET.SubElement(parent, key)
                build_xml(child, value)
    else:
        parent.text = data

# Пример

file_path = r"C:\Users\79125\OneDrive\Рабочий стол\Учёба ПНИПУ АСУ\4 семестр\Методы и алгоритмы обработки слабоструктурированных данных\lr2\sample1.html" 
output_file_path = r"C:\Users\79125\OneDrive\Рабочий стол\Учёба ПНИПУ АСУ\4 семестр\Методы и алгоритмы обработки слабоструктурированных данных\lr2"

input_format = detect_input_format(file_path)
output_format = detect_output_format(output_file_path)

if input_format is None:
    print("Не удалось определить формат входного файла.")
elif output_format is None:
    print("Не удалось определить формат выходного файла.")
else:
    content = load_file(file_path)

    if input_format == 'html':
        if output_format == 'json':
            result = html_to_json(content)
        elif output_format == 'xml':
            result = html_to_xml(content)
        else:
            result = "Формат вывода не поддерживается для HTML."
    elif input_format == 'xml':
        if output_format == 'json':
            result = xml_to_json(content)
        elif output_format == 'csv':
            result = xml_to_csv(content)
        else:
            result = "Формат вывода не поддерживается для XML."
    elif input_format == 'json':
        if output_format == 'xml':
            result = json_to_xml(content)
        elif output_format == 'csv':
            result = json_to_csv(content)
        else:
            result = "Формат вывода не поддерживается для JSON."
    elif input_format == 'csv':
        if output_format == 'json':
            result = csv_to_json(content)
        elif output_format == 'xml':
            result = csv_to_xml(content)
        else:
            result = "Формат вывода не поддерживается для CSV."
    else:
        result = "Не удалось определить формат данных."

    if result:
        save_to_file(result, output_file_path)
        print("Результат успешно сохранен.")
    else:
        print("Произошла ошибка при обработке данных.")

Не удалось определить формат выходного файла.
